In [13]:
from sqlalchemy import create_engine

import env
import certifi
from pymongo import MongoClient
import pandas as pd

ca = certifi.where()

In [14]:
# pull stock_info from neondb
engine = create_engine(env.NEON_DB_CONSTR)
stock_info = pd.read_sql('SELECT * FROM stock_info', engine)

In [19]:
watchlist = stock_info[['Symbol']].rename(columns={'Symbol': 'symbol'})
watchlist['interval'] = '1d'
watchlist['data_source'] = 'yfinance'
watchlist['market_index'] = 'SPY'
watchlist['auto_synced'] = True

In [20]:
_mongo_client = MongoClient(env.MONGO_URL, tlsCAFile=ca)
_db = _mongo_client['asset-tracking']
_collection = _db['watchlist']
send_watchlist = watchlist.to_dict('records')
# insert watchlist into mongo as a single document:
# {
#     "username": "bjahnke",
#     "watchlist": [
#         {
#             "symbol": "AAPL",
#             "interval": "1d",
#             "data_source": "yfinance",
#             "market_index": "SPY"
#         },
#         {
#             "symbol": "MSFT",
#             "interval": "1d",
#             "data_source": "yfinance",
#             "market_index": "SPY"
#         }
#     ]
# }
_collection.insert_one({
    "username": "bjahnke71",
    "watchlist": send_watchlist
})



In [1]:
_mongo_client = MongoClient(env.MONGO_URL, tlsCAFile=ca)
_db = _mongo_client['asset-tracking']
_collection = _db['watchlist']

_latest_entry = _collection.find_one({"username": "bjahnke"}, sort=[("_id", -1)])
_watchlist = {
    'event_id': _latest_entry.get('event_id'),
    'venue_id': _latest_entry.get('venue_id'),
    'performer_id': _latest_entry.get('performer_id')
}

class MongoWatchlistClient:
    def __init__(self, mongo_client, db: str, watchlist_collection: str):
        self._mongo_client = mongo_client
        self._db = _mongo_client[db]
        self._collection = self._db[watchlist_collection]

    def get_watchlist(self):
        return self._collection.find_one({"username": "bjahnke71"}, sort=[("_id", -1)])


SyntaxError: incomplete input (3613436695.py, line 11)